<a href="https://colab.research.google.com/github/visha1Sagar/Text-To-SQL/blob/main/Qwen2_5_7B_semEval_text_to_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing libraries

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

### Model

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

#### Fine tune the text to sql model

In [3]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "Vishal-Sagar/Qwen2.5_7B-text-to-sql-basic",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.12: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 24, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [5]:
from datasets import load_dataset
import json
from datasets import Dataset

alpaca_prompt = """
### Instruction:
You are a DataFrame Query Expert. Your task is to write accurate and efficient Pandas DataFrame queries based on the given schema and user question. Adhere to these guidelines:

Input Format:
- Schema: A detailed representation of the table(s) in a Pandas DataFrame format, including column names and data types.
- Question: A natural language question to be answered using the schema.

Output Requirements:
- Pandas Query Only: Return only the Pandas query without any additional text or explanation.
- Query Style: Ensure queries are concise, optimized for readability, and adhere to Pandas best practices.

Example Input-Output Pairs:
Input:
Schema: Table Name: head (age (int64))
Question: "How many heads of the departments are older than 56?"

Output:
head[head['age'] > 56].shape[0]


Input:
Schema: Table Name: head (name (object), born_state (object), age (int64))
Question: "List the name, born state, and age of the heads of departments ordered by age."

Output:
head.sort_values(by='age', ascending=True)[['name', 'born_state', 'age']]


### Input:
Question: {input}
Schema: {context}

### Response:
{output}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):

    inputs       = examples["Input"]
    outputs      = examples["Pandas Query"]

    texts = []
    for input, output in zip(inputs, outputs):

        input = input.split("\n")
        context      = input[-1]
        input = "\n".join(input[:-1])
        input, context = context, input
        output = "".join(output.split("=")[1:])

        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input=input, context=context, output=output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


# from datasets import load_dataset
dataset = load_dataset("Rahima411/text-to-pandas", split = "train")


dataset = dataset.map(formatting_prompts_func, batched = True,)

In [6]:
print(dataset[0]['text'])


### Instruction:
You are a DataFrame Query Expert. Your task is to write accurate and efficient Pandas DataFrame queries based on the given schema and user question. Adhere to these guidelines:

Input Format:
- Schema: A detailed representation of the table(s) in a Pandas DataFrame format, including column names and data types.
- Question: A natural language question to be answered using the schema.

Output Requirements:
- Pandas Query Only: Return only the Pandas query without any additional text or explanation.
- Query Style: Ensure queries are concise, optimized for readability, and adhere to Pandas best practices.

Example Input-Output Pairs:
Input:
Schema: Table Name: head (age (int64))
Question: "How many heads of the departments are older than 56?"

Output:
head[head['age'] > 56].shape[0]


Input:
Schema: Table Name: head (name (object), born_state (object), age (int64))
Question: "List the name, born state, and age of the heads of departments ordered by age."

Output:
head.sor

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 100 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 2,
        max_steps=100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/57509 [00:00<?, ? examples/s]

In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.129 GB of memory reserved.


In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 57,509 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 100
 "-____-"     Number of trainable parameters = 60,555,264


Step,Training Loss
1,2.241200
2,2.263500
3,2.006500
4,1.609700
5,1.280000
6,0.945300
7,0.751800
8,0.612900
9,0.535800
10,0.456600


In [10]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1572.445 seconds used for training.
26.21 minutes used for training.
Peak reserved memory = 8.082 GB.
Peak reserved memory for training = 1.953 GB.
Peak reserved memory % of max memory = 54.801 %.
Peak reserved memory for training % of max memory = 13.242 %.


<a name="Inference"></a>
### Inference
Let's run the model!

In [16]:
qs = "What is the name of the shipping agent of the document with id 2?"
table_schema = "Table Name: Documents (Id (object)) Table Name: Ref_Document_Status (Id (object))"

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def run_model(question, context):

    inputs = tokenizer(
    [
        alpaca_prompt.format(
            input=question, # question
            context=context, # context database schema
            output="", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
    decoded_output = tokenizer.batch_decode(outputs)

    return decoded_output[0].split("### Response:\n")[-1].split("<|endoftext|>")[0]

output = run_model(qs, table_schema)

In [17]:
print(output)


### Instruction:
You are a DataFrame Query Expert. Your task is to write accurate and efficient Pandas DataFrame queries based on the given schema and user question. Adhere to these guidelines:

Input Format:
- Schema: A detailed representation of the table(s) in a Pandas DataFrame format, including column names and data types.
- Question: A natural language question to be answered using the schema.

Output Requirements:
- Pandas Query Only: Return only the Pandas query without any additional text or explanation.
- Query Style: Ensure queries are concise, optimized for readability, and adhere to Pandas best practices.

Example Input-Output Pairs:
Input:
Schema: Table Name: head (age (int64))
Question: "How many heads of the departments are older than 56?"

Output:
head[head['age'] > 56].shape[0]


Input:
Schema: Table Name: head (name (object), born_state (object), age (int64))
Question: "List the name, born state, and age of the heads of departments ordered by age."

Output:
head.sor

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model.

In [18]:
from google.colab import userdata

In [ ]:
# model.save_pretrained("lora_model") # Local saving
# tokenizer.save_pretrained("lora_model")
model.push_to_hub("Vishal-Sagar/Qwen2.5_7B-text-to-df_query", token = userdata.get('HF_TOKEN')) # Online saving
tokenizer.push_to_hub("Vishal-Sagar/Qwen2.5_7B-text-to-df_query", token = userdata.get('HF_TOKEN')) # Online saving

README.md:   0%|          | 0.00/579 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Vishal-Sagar/Qwen2.5_7B-text-to-df_query


In [ ]:
# # prompt: make zip folder of lora_model and download it

# !zip -r lora_model.zip lora_model
# !ls
# from google.colab import files
# files.download("lora_model.zip")

  adding: lora_model/ (stored 0%)
  adding: lora_model/adapter_config.json (deflated 56%)
  adding: lora_model/tokenizer.json (deflated 85%)
  adding: lora_model/README.md (deflated 66%)
  adding: lora_model/tokenizer_config.json (deflated 84%)
  adding: lora_model/tokenizer.model (deflated 55%)
  adding: lora_model/special_tokens_map.json (deflated 76%)
  adding: lora_model/added_tokens.json (deflated 62%)
  adding: lora_model/adapter_model.safetensors (deflated 8%)
huggingface_tokenizers_cache  lora_model  lora_model.zip  outputs  sample_data


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>